<a href="https://colab.research.google.com/github/Melikakmm/GPR_fitting_interactive_potential/blob/main/fitpot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Process regression for fitting interatomic potentials

## Workshop Aims
* Data representation using invariant descriptors
* Building covariance matrices for derived quantities - learning from total energy data
* Uncertainty analysis of predicted energy values
* Optimising hyperparameters
* Relaxing geometry using a machine learned potential



In [3]:
!alias gcc='gcc-11'
!alias cc='gcc-11'
!alias g++='g++-11'
!alias c++='c++-11'

In [ ]:
pip install GPy

In [ ]:
pip install ase

In [ ]:
pip install nglview

In [11]:
import numpy as np
import pandas as pd
import GPy

from ase.io import read
import nglview

from IPython.display import display

%pylab inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

Populating the interactive namespace from numpy and matplotlib


In [13]:
plt.rcParams["figure.figsize"] = (10, 8)

## Part 1
### Learning the interaction energy of a single water molecule

The quantum mechanical energy of a water molecule - within the Born-Oppenheimer approximation - is a function of the geometry of the molecule, i.e. the Cartesian coordinates $\mathbf{r}_\rm{O}$, $\mathbf{r}_{\rm H_1}$ and  $\mathbf{r}_{\rm H_2}$. Of course, this energy does not depend on the orientation of the water molecule if there is no external field interacting with the molecule.

<img src="water.png">

We can rewrite the energy function as $E(\mathbf{r}_{\rm O},\mathbf{r}_{\rm H_1},\mathbf{r}_{\rm H_2}) \equiv E(r_{\rm OH_1},r_{\rm OH_2},\theta_{\rm HOH})$ where $r_{\rm OH}$ are the bond lengths and $\theta_{\rm HOH}$ the bond angle. This description is invariant to rotations and translations. We also know that the 'label' of the hydrogens is unimportant (i.e. swapping $\rm H_1$ and $\rm H_2$ does not change the energy), therefore it is useful to symmetrise the distances.

In this exercise, we will use the following data coordinates:
* $r_+ = r_{\rm OH_1} + r_{\rm OH_2}$
* $r_- = (r_{\rm OH_1} - r_{\rm OH_2})^2$
* $a = \mathbf{r}_{\rm OH_1} \cdot \mathbf{r}_{\rm OH_2}$

### Question 1

**Explain why the descriptor functions $r_+$, $r_-$ and $a$ are invariant to rigid rotations, translations of the molecule and permutations of the hydrogen indices.**

answer: due to the symmetry of the water molecule around the oxygen axis, $r_+$, and $r_-$ are both invariant against any translation and rotation. Moreover, because $\mathbf{r}_{\rm OH_1}$ and $\mathbf{r}_{\rm OH_2}$ vectors are relative positional vectors and are not coming from the origin of the coordinates, they are also invariant to rigid rotation and translation of the molecule.






YOUR ANSWER HERE

In [15]:
# Read in water configurations
water_configs = read("water_configs.xyz", format="extxyz", index=":")
# There should be 2886 independent water molecules
print(len(water_configs))

2886


In [21]:
water_configs[0:10]

[Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator(...)),
 Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator(...)),
 Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator(...)),
 Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator(...)),
 Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator(...)),
 Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator(...)),
 Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator(...)),
 Atoms(symbols='OH2', pbc=True, cell=[30.0, 30.0, 30.0], map_shift=..., n_neighb=..., calculator=SinglePointCalculator

In [16]:
# optional - for visualisation
nglview.show_asetraj(water_configs)

NGLWidget(max_frame=2885)

In [41]:
for a in water_configs[0:1]:
    p = a.get_positions()
    print(a.get_distance(0, 2))


0.6000000019741349


In [42]:
x = [] # input features
y = [] # target values

# Calculating descriptor vectors for each molecule and collect target energies
for a in water_configs:
    p = a.get_positions() # positions: O, H1, H2
    rOH1 = a.get_distance(0, 1)# calculating distance OH1
    rOH2 = a.get_distance(0, 2)# calculating distances OH2
    aHOH = np.dot(p[1]-p[0], p[2]-p[0])# dot product between the vectors OH1 and OH2
    # <Here rOH1 and rOH2 are only distances and aHOH is a dot product of two vectors.>
    x.append([(rOH1 + rOH2), (rOH1 - rOH2)**2, aHOH]) # collect descriptor vectors

    y.append(a.get_potential_energy()) # QM energy of a water molecule

We randomly split the data to a train and test set and then use it to train a Gaussian process regression model using `GPy`.

In [ ]:
from sklearn.model_selection import train_test_split

# test_size sets the fraction of the test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.9)

X_train = np.array(X_train)
y_train = np.array(y_train)[:, None] # GPy needs a 2D array

In [ ]:
# input_dim: dimensionality of data points - our descriptor vectors have three elements
# variance: range (squared) of the function
print("Range of input data: {:.2f}".format(y_train.max()-y_train.min()))
# lengthscale: characteristic lengthscale - start with a default of 0.5
# ARD: automatic relevance determination - allows separate lengthscales for each three descriptor dimensions

kernel = GPy.kern.RBF(input_dim=3, variance=1., lengthscale=10.,ARD=True)

### Question 2.a

**Why is it useful to use 'automatic relevance determination' (separate lengthscales, rather than one for all three dimensions) in this situation?** [5 marks]

YOUR ANSWER HERE

In [ ]:
# Generate a Gaussian Process model by adding training data. Start with a small noise parameter.
m = GPy.models.GPRegression(X_train, y_train, kernel, noise_var=1.)

In [ ]:
# Optimise the likelihood with respect of the hyperparameters of the model
m.optimize_restarts(num_restarts = 10);

### Question 2.b

**When optimising the hyperparameters, what do we mean by restarts and why is it useful to do more than one?** [5 marks]

YOUR ANSWER HERE

In [ ]:
# Print the result of the likelihood optimisation
display(m)
m.rbf.lengthscale

`m` contains the trained model: a combination of the _prior_ (in the form of the kernel and its hyperparameters, as well as the noise model) and _data_ (in the form of geometrical descriptors and corresponding energy values).

### Question 2.c

**From the output of the previous cell, what is the noise variance of the optimal model? What does this tell us about the data?** [5 marks]

YOUR ANSWER HERE

In [ ]:
# Use the model to predict the energy of water configurations.

y_train_predict, y_train_error = m.predict(np.array(X_train)) # predict energies of training configurations
y_test_predict, y_test_error =   # predict energies of the test set configurations

# root-mean-square error (RMSE)
rmse =
print("RMSE = {:.3f} eV".format(rmse))

### Question 2.d

**Plot graphs to show the correlation of actual and predicted energies, and of the actual and predicted error** [5 marks]

In [ ]:
# Plot the correlation of actual and predicted data
plt.plot(y_train,y_train_predict,"s", label='Train')
plt.errorbar(y_test,y_test_predict,yerr=np.sqrt(y_test_error[:,0]),fmt=".", label='Test')
plt.xlabel('True energies / eV')
plt.ylabel('Predicted energies / eV')
plt.legend()
plt.show()

# Plot the correlation of actual and predicted error
plt.plot()
plt.xlabel('True error / eV')
plt.ylabel('Predicted error / eV')
plt.show()

### Question 3.a

**Briefly disucss how accurate the Gaussian Process water model is. How accurate is the error prediction?** [5 marks]

YOUR ANSWER HERE

### Question 3.b

We only used 10% of the data for training. By adapting the code above, starting from where we split the data into test and train set, fit further GP models with different fractions (10%, 20%, 40%, 80%) of training data - don't forget to optimise the likelihood. Summarise in a table how the root mean squared error (RMSE) changes. Write a short paragraph about the advantages and disadvantages of using more training data. [5 marks code + 5 marks discussion]

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=['train_size', 'RMSE'])

df['train_size'] = [0.1, 0.2, 0.4, 0.8]

for i, train_size in enumerate(df.train_size):
    # split data
    X_train = np.array(X_train)
    y_train = np.array(y_train)[:, None]
    # create modeel
    # optimise
    # predict on test set y_test_predict, y_test_error =
    rmse =
    print(train_size, rmse)
    df.iloc[i, 1] = rmse

df

YOUR ANSWER HERE

### Optimising the geometry of the water molecule, using a Gaussian Process energy model
In the following exercise, we will relax the Cartesian coordinates of a water molecule, in order to find the equilibrium geometry. We will use a wrapper function that generates the symmetrised descriptors and using the GP model, predicts the energy of the water molecule. For simplicity, we use the Nelder-Mead algorithm to find the minimum of the function, as it does not require access to the gradients.

In [ ]:
# This is the wrapper function: Cartesian coordinates -> symmetrised descriptors -> GP
def GP_water_energy(x):
    _x = np.reshape(x, (3,3)) # Input is a flat array of x,y,z coordinates of three atoms.
    r1 = np.linalg.norm(_x[0] - _x[1]) # rOH1
    r2 = np.linalg.norm(_x[0] - _x[2]) # rOH2
    a = np.dot(_x[0] - _x[1],
               _x[0] - _x[2]) # dot product of the two OH vectors
    _xx = np.array([[(r1 + r2),
                     (r1 - r2)**2,
                     a]]) # symmetrised descriptor (see above)
    res = m.predict(_xx)[0][0][0] # GP prediction
    return res

We start from a not-too crazy initial condition, and relax the position of all atoms.

In [ ]:
from scipy.optimize import minimize

x0 = np.array([0,0,0,       # Oxygen atom
               1.2,0,0,     # 1st hydrogen atom
               -0.5,0.7,0]) # 2nd hydrogen atom

res = minimize(GP_water_energy, # function to minimise
               x0,              # initial condition
               method='nelder-mead',
               options={"maxiter":1000, # number of iterations in the minimiser
                        "fatol":1e-6    # stopping criterion (tolerance)
                       })
# res.x contains the final configuration

In [ ]:
# We transform the Cartesian coordinates into bond lengths and the bond angle,
# for easier interpretation

_x = np.reshape(res.x, (3, 3))
r1 =
r2 =
a =
print("r1 = {:.3f} A\nr2 = {:.3f} A\nangle = {:.1f} degrees".format(r1,r2,np.rad2deg(np.arccos(a))))

### Question 4

**Inspect the resulting water geometry. How realistic is it? Compare the geometrical parameters to literature (experimental and/or theoretical) values.** [5 marks]

YOUR ANSWER HERE

### Extension question (not assessed)
Try different kernels `GPy.kern.` in the Gaussian Process regression.

In [ ]:
# YOUR ANSWER HERE

## Part 2
### Inferring a pair interaction model based on total energy information

For certain types of atomic systems, the total, inherently many-body interaction energy may be approximated by pair interaction terms: $E(\mathbf{r}_1, \mathbf{r}_2, \ldots \mathbf{r}_N) \approx \sum^N_{i<j} E_2(r_{ij})$, where the system consists of $N$ atoms with Cartesian coordinates $\mathbf{r}_1, \mathbf{r}_2, \ldots \mathbf{r}_N$. $E_2$ is a function of interatomic distances $r_{ij}$. Example where this works well are noble gases or certain types of metals.

In this part of the workshop, we will study a toy problem, where the total interaction energy consists explicitly of two-body terms, and the aim of the workshop is to recover the original model. We will use configurations of clusters of 12-19 atoms, and the interaction energy (the underlying function we are trying to fit) will be the Lennard-Jones model for the total energy

$$
E_2^{LJ}(r_{ij}) = 4\left(\frac1{r_{ij}^{12}} - \frac1{r_{ij}^6} \right)
$$

In [ ]:
# We load the cluster configurations
cluster_trajectory = read("clusters.xyz", index=":")

In [ ]:
# optional
nglview.show_asetraj(cluster_trajectory)

In [ ]:
# For each cluster, we calculate the total energy and collect it in cluster_energy
from ase.calculators.lj import LennardJones
p = LennardJones()

cluster_energy = []
for a in cluster_trajectory:
    a.set_calculator(p)
    cluster_energy.append(a.get_potential_energy())

In [ ]:
# We split the data in train and test set.
# You may adjust the ratio by varying "test_size".

from sklearn.model_selection import train_test_split

cluster_train, cluster_test, cluster_energy_train, cluster_energy_test = train_test_split(
    cluster_trajectory, cluster_energy, test_size=0.4)

The model we use for the total energy is

$$E = \sum_{i<j} {\mathcal GP}(r_{ij}).$$

To fit the GP, we need to obtain all pair-wise distances in each configuration.

In [ ]:
from ase.neighborlist import neighbor_list

def get_distances(atoms_array,cutoff=3.0):
    distances = []
    for a in atoms_array:
        i, j, d = neighbor_list('ijd', a, cutoff)
        d = d[i < j]  # We exclude duplicates
        distances.append(d)
    return distances # List of list of distances

In [ ]:
distances_train = get_distances(cluster_train)
distances_test =

The target data is the sum of Gaussian Process models. We need to find the covariance of total energies: if the covariance of two pair interactions terms is $k(r,r')$, the covariance of two total energies of two configurations $A$ and $B$ is the sum of covariance functions:

$$\langle E_A E_B \rangle = \sum_{ij \in A, i'j' \in B} k(r_{ij},r_{i'j'})$$

The following class implements this idea.

In [ ]:
from scipy.spatial.distance import cdist

class SumGP:
    """Class implementing a sum Gaussian Process"""

    def __init__(self, length_scale=1.0, function_range=1.0, error=0.1):
        """Initialise the class with the GP hyperparameters"""
        self.length_scale = length_scale
        self.function_range = function_range
        self.error = error

    def update_covariance(self):
        """Update the covariance matrix"""
        self.covariance = self.sum_covariance(self.training_data,self.training_data)
        self.covariance += np.eye(len(self.covariance))*self.error**2

    def update_weights(self):
        """Update the GP model weights"""
        self.weights = np.linalg.solve(self.covariance,self.training_target)

    def update_gp(self):
        """Update the covariance and the GP model weights"""
        self.update_covariance()
        self.update_weights()

    def set_training_data(self,distances,energies):
        """Add training data and perform the fit"""
        self.training_data = distances
        self.training_target = energies
        self.update_gp()

    def set_parameters(self,length_scale=None,function_range=None,error=None):
        """Update GP hyperparameters"""
        changed = False
        if length_scale is not None:
            self.length_scale = length_scale
            changed = True
        if function_range is not None:
            self.function_range = function_range
            changed = True
        if error is not None:
            self.error = error
            changed = True
        if changed:
            self.update_gp()

    def predict(self,distances, do_variance=False, do_covariance=False):
        """
        Predicts total energy for a set of pairwise distances,
        optionally with variances
        """
        k = self.sum_covariance(distances,self.training_data)
        mean = np.dot(k,self.weights)
        res = {"mean":mean}
        if do_variance:
            variance = []
            for _d,_k in zip(distances,k):
                variance.append(self.error**2+
                    self.sum_covariance([_d],[_d])[0,0] -
                           np.dot(_k, np.linalg.solve(self.covariance,_k))
                )
            res["variance"] = variance

        if do_covariance:
            covariance = self.sum_covariance(distances,distances)
            res["covariance"] = ( np.eye(len(distances))*self.error**2 +
                                 covariance - np.dot(k,np.linalg.solve(self.covariance,k.T)) )

        return res

    def sum_covariance(self, d1, d2):
        """Sum covariance function"""
        covariance = []
        for i, _d1 in enumerate(d1):
            _c_row = []
            for j, _d2 in enumerate(d2):
                distance_matrix = cdist(
                    np.array(_d1)[:,None],
                    np.array(_d2)[:,None],
                    'euclidean')

                _c = np.sum(self.function_range**2*np.exp(
                            -0.5*distance_matrix**2/self.length_scale**2))
                _c_row.append(_c)
            covariance.append(_c_row)
        covariance = np.array(covariance)
        return covariance


We now use this class to build a Gaussian Process regression model

In [ ]:
# Initialise the GP with hyperparameters
sum_gp = SumGP(length_scale=0.25, error=0.5, function_range=5.0)

In [ ]:
# Add data and fit the model
sum_gp.set_training_data(distances_train,cluster_energy_train)

### Optimising the hyperparameters
To find the optimal hyperparameters (`length_scale`, `error` and `function_range`) we can try maximising the likelihood of the model:

$$
\log L = -\frac{1}{2} \mathbf{y}^T \mathbf{K}^{-1} \mathbf{y} - \frac{1}{2} \log |\mathbf{K}| - \frac{n}{2} \log 2 \pi$$

Remember, the two main components of this expression balance the goodness of the fit (first term) and the complexity of the model (second term).

### Question 5.a

Implement a log likelihood function for the `SumGP` class using the template provided below. Use the numpy function [np.linalg.slogdet](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.slogdet.html) to compute the log of the determinent of the covariance matrix. For the first term of the log likelihood expression, notice that $\mathbf{K}^{-1} \mathbf{y} = \mathbf{w}$, the fitted weights of the model, stored as `self.weights` in the `SumGP` class. [10 marks]


In [ ]:
def get_likelihood(self):
    """Calculate GP likelihood"""

    res = 0.0
    ### BEGIN SOLUTION
    ### END SOLUTION

    return res

SumGP.get_likelihood = get_likelihood

### Question 5.b

Compute the log likelihood as a function of the `length_scale` hyperparameter. Plot the results, and determine the optimal parameter, given the other two hyperparameters remain fixed. [10 marks]

In [ ]:
sum_gp =
# Add data and fit the model

length_scale_array = # np.linspace()
likelihood_array = []
for _l in length_scale_array:
    sum_gp.set_parameters(length_scale=_l)
    likelihood_array.append(sum_gp.get_likelihood())

plt.plot()
plt.xlabel('Length scale')
plt.ylabel('Log likelihood');

### Question 6.a

Set the `length_scale` hyperparameter to the optimal value you found above, and plot the two-body interatomic potential model as a function of interatomic distances, with error bars, and compare it with the known solution, the Lennard-Jones model. You need to set the range of distances where you want to inspect the model - start from short distances and go beyond the cutoff (3.0) we used. [10 marks]

Here is some sample code to do predictions to get you started

```python
# Predict pair interaction energies using the GP model
r_min = ...
r_max = ...
r_test = np.linspace(r_min,r_max)[:,None]
test_gp = sum_gp.predict(r_test,do_variance=True,do_covariance=True)
e_test_gp = test_gp["mean"]
v_test_gp = test_gp["variance"]
c_test_gp = test_gp["covariance"]
```


In [ ]:
sum_gp.set_parameters(length_scale=)

# Lennard-Jones model - the target of the GP model
e_test_lj = # function of r_test

# Predict pair interaction energies using the GP model
test_gp = sum_gp.predict(r_test,do_variance=True,do_covariance=True)
e_test_gp = test_gp["mean"]
v_test_gp = test_gp["variance"]
c_test_gp = test_gp["covariance"]

# Plot the results
plt.plot(r_test,e_test_lj)
plt.errorbar(r_test,e_test_gp,yerr=np.sqrt(v_test_gp))
plt.ylim([-1.2,4])
plt.xlabel('Distance $r_{ij}$')
plt.ylabel('Energy / eV')

### Question 6.b

Inspect the predicted error. Identify the domains where the predicted error is large - what is the reason for this? [10 marks]

YOUR ANSWER HERE

### Question 6.c

In the code cell below, using the `np.random.multivariate_normal` function (as seen in the lecture), with `test_gp["mean"]` as the mean and `test_gp["covariance"]` as the covariance, where `test_gp` is the result of calling `sum_gp.predict()`), draw samples from the posterior. Plot the results together with the known target function (Lennard-Jones pairwise interactions). Looking at these samples, how appropriate is our prior (squared exponential kernel)? Which feature of the Lennard-Jones model is causing problems? [5 marks code + 5 marks discussion]

In [ ]:
N_sample = 5

for i in range(N_sample):
    s_test_gp = # np.random.multivariate_normal()
    plt.plot(r_test, s_test_gp, 'r--')
plt.plot(r_test,e_test_lj,"k-")
plt.ylim([-1.2,4])
plt.show()

YOUR ANSWER HERE

### Question 7

Adapt the code used above to predict GP energies (i.e., using `sum_gp.predict()`), evaluate the GP model on the test set `distances_test` we generated by splitting the original data set. Plot the correlation of the target energy and the prediction, with error bars. [5 marks]

In [ ]:
cluster_gp = sum_gp.predict(distances_test, do_variance=True)
cluster_energy_predict =
cluster_energy_variance =

plt.errorbar(cluster_energy_test, cluster_energy_predict,
             yerr=np.sqrt(cluster_energy_variance),fmt=".")
plt.xlabel('True energy / eV')
plt.ylabel('Predicted energy / eV');

### Extension  question (not assessed)

Instead of the squared exponential kernel, implement another kernel (e.g. the exponential kernel $k(r,r') = \exp ( |r-r'| / l)$ or any other) in the `SumGP` class. Does it represent a better prior than the squared exponential kernel?